# COMP9318 Lab2

## Instructions
1. This note book contains instructions for **COMP9318-lab2**.

* You are required to complete your implementation in a file `submission.py` provided along with this notebook.

* You are not allowed to print out unnecessary stuff. We will not consider any output printed out on the screen. All results should be returned in appropriate data structures return by corresponding functions.

* You need to submit the code for **lab2** via following link: http://kg.cse.unsw.edu.au:8318/lab2/ .

* For each question, we have provided you with detailed instructions along with question headings. In case of any problem, you can post your query @ Piazza.

* If you choose to skip a question, leave the corresponding function body as it is (i.e., keep the `pass` line), otherwise it may affect your mark for other questions.

* You are allowed to add other functions and/or import additional modules (you may have to in this lab), but you are not allowed to define global variables. **Only functions are allowed** in `submission.py`. 

* You should not import unnecessary modules/libraries, failing to import such modules at test time will lead to errors.

* We will provide immediate feedback on your submission. You can access your scores using the online submission portal on the same day. 

* For **Final Evaluation** we will be using a different dataset, so your final scores may vary.  

* You are allowed to submit as many times as you want before the deadline, but **ONLY the latest version will be kept and marked**.

* Submission deadline for this assignment is **11:59:59 PM on 4th April, 2018**. We will **not** accept any late submissions.

# Question 1: Optimized BUC algorithm (50 points)

You need to implement the full `buc_rec_optimized` algorithm with the single-tuple optimization (as described below). Given an input dataframe:

 A | B | M 
---|---|---
 1 | 2 | 100
 2 | 1 | 20

Invoking  `buc_rec_optimized` on this data will result in following dataframe: 


 A | B | M
---|---|---
 1 | 2 | 100
 1 |ALL| 100
 2 | 1 | 20 
 2 |ALL| 20
ALL| 1 | 20
ALL| 2 | 100
ALL|ALL| 120

In the file `submission.py`, we have pre-defined the `buc_rec_optimized` function and its helper functions. 

## Input and output

Both input and output are dataframes.

The input dataframe (i.e., the base cuboid) is directly generated from the input file. Given the dimensionality of the base cuboid is $d$, each row is like:

<pre>
v_1  v_2 ...  v_d  m
</pre>

where v_i is the cell's value on the i-th dimension, and m is the measure value. 

The output dataframe contains $n$ rows, each for a non-empty cell in the compute data cube derived from the input base cuboid. Each row is formatted like input:

<pre>
v_1  v_2 ...  v_d  m
</pre>

where v_i is the cell's value on the i-th dimension, and m is the measure value. 


## The single-tuple optimization

Consider the naive way of recursive implementation of the BUC algorithm, you will notice that it uses several recursive calls to compute all the derived results from an input that consists of only one tuple. This is certainly a waste of computation. 

For example, if we are asked to compute the cube given the following input

 B | C | M 
---|---|---
 1 | 2 | 100

We can immmediately output the following, **without** using any recursive calls. 

<pre>
1    2    100
*    2    100
1    *    100
*    *    100
</pre>



** Note: For lab-2, you are allowed to use only two libraries, i.e., pandas, and numpy.** 

In [1]:
import pandas as pd
import numpy as np

In [2]:
##============================================================
# Data file format: 
# * tab-delimited input file
# * 1st line: dimension names and the last dimension is assumed to be the measure
# * rest of the lines: data values.

def read_data(filename):
    df = pd.read_csv(filename, sep='\t')
    return (df)
      
# helper functions
def project_data(df, d):
    # Return only the d-th column of INPUT
    return df.iloc[:, d]

def select_data(df, d, val):
    # SELECT * FROM INPUT WHERE input.d = val
    col_name = df.columns[d]
    return df[df[col_name] == val]

def remove_first_dim(df):
    # Remove the first dim of the input
    return df.iloc[:, 1:]

def slice_data_dim0(df, v):
    # syntactic sugar to get R_{ALL} in a less verbose way
    df_temp = select_data(df, 0, v)
    return remove_first_dim(df_temp)

In [3]:
def buc_rec_optimized(df):  # do not change the heading of the function

    if df.shape[0] == 1:
        df_out = one_dim_duc(df)
    else:
        header = list(df)
        df_out = pd.DataFrame(columns=header)
        _buc_rec_optimized(df, [], df_out)
    return df_out


def deepcopy(list):
    b = []
    for i in list:
        b.append(i)
    return b


def read_data(filename):
    df = pd.read_csv(filename, sep='\t')
    return (df)


def output(val):
    print('=>\t{}'.format(val))


def one_dim_duc(df):
    vals = list(df.loc[0])
    result = [vals[:-1]]
    for i, val in enumerate(result):
        temp = deepcopy(val)
        for j, list_val in enumerate(temp):
            if list_val != 'ALL':
                temp2 = deepcopy(temp)
                temp2[j] = 'ALL'
                if temp2 not in result:
                    result.append(temp2)

    for i in result:
        i.append(vals[-1])
    result = pd.DataFrame(result, columns=list(df))
    return result


def _buc_rec_optimized(df, pre_num, df_out):  # help function
    # Note that input is a DataFrame
    dims = df.shape[1]

    if dims == 1:
        # only the measure dim
        input_sum = sum(project_data(df, 0))
        pre_num.append(input_sum)

        df_out.loc[len(df_out)] = pre_num

    else:
        # the general case

        dim0_vals = set(project_data(df, 0).values)
        temp_pre_num = deepcopy(pre_num)
        for dim0_v in dim0_vals:
            pre_num = deepcopy(temp_pre_num)
            sub_data = slice_data_dim0(df, dim0_v)
            pre_num.append(dim0_v)

            _buc_rec_optimized(sub_data, pre_num, df_out)
        ## for R_{ALL}
        sub_data = remove_first_dim(df)

        pre_num = deepcopy(temp_pre_num)
        pre_num.append("ALL")
        _buc_rec_optimized(sub_data, pre_num, df_out)

input_data = read_data('./asset/a.txt')


output=buc_rec_optimized(input_data)
output

,A,B,C,M
0,1,2,3,100
1,1,2,ALL,100
2,1,ALL,3,100
3,1,ALL,ALL,100
4,2,1,3,20
5,2,1,ALL,20
6,2,ALL,3,20
7,2,ALL,ALL,20
8,ALL,1,3,20
9,ALL,1,ALL,20


In [4]:

################# Question 1 #################

# you can call helper functions through the helper module (e.g., helper.slice_data_dim0)


        
def single(df):
    all_row=[]
    enulist=[]
    dim=len(df.index)
    
    for i in range(dim):
        single_tuple=df.iloc[i]
        x=[]
        for j in single_tuple:
            x.append(j)
        enulist.append(x)

    for i, oli in enumerate(enulist):
        row=oli[:-1]
        if oli not in all_row:
            all_row.append(oli)

        for i2 in range(len(row)):
            new_row=oli[:]
            if new_row[i2]!="ALL":
                new_row[i2]="ALL"
                if new_row not in enulist:
                    enulist.append(new_row)
    
    
    print()
    for i1, i in enumerate(all_row):
        for i2, j in enumerate(all_row[i1+1:]):
            i3=i2+i1+1
            if i[:-1]==j[:-1]:
                i[-1]=i[-1]+j[-1]
                all_row.pop(i3)
              
     
    
    
    result = pd.DataFrame(data=all_row, columns=list(df))
    return result

    
input_data = read_data('./asset/a.txt')


output=single(input_data)
output

,A,B,C,M
0,1,2,3,100
1,2,1,3,20
2,ALL,2,3,100
3,1,ALL,3,100
4,1,2,ALL,100
5,ALL,1,3,20
6,2,ALL,3,20
7,2,1,ALL,20
8,ALL,ALL,3,120
9,ALL,2,ALL,100


In [6]:
## You can test your implementation using the following code...

import submission as submission
input_data = read_data('./asset/aa.txt')
output = submission.buc_rec_optimized(input_data)
output

,A,B,M
0,1,2,100
1,1,ALL,100
2,2,1,20
3,2,ALL,20
4,ALL,1,20
5,ALL,2,100
6,ALL,ALL,120


# Question 2: Optimal binning algorithm using dynamic programming (50 points)

You need to implement the optimal binning algorithm using the dynamic programming algorithm we discussed in the lecture. You are allowed to use $O(n^2)$ space. 

## Input

The input contains data (in a list) and the number of bins (an integer).

## Output

You are required to output the binning result and the matrix computed by the algorithm.

The matrix entries record optimal binning cost for a suffix of the input array using a certain number of bins. You should assign -1 to all the invalid solutions.

In [2]:
x = [3, 1, 18, 11, 13, 17]
num_bins = 4

In [6]:
import numpy as np

def sse(arr):
    if len(arr) == 0: # deal with arr == []
        return 0.0

    avg = np.average(arr)
    val = sum( [(x-avg)*(x-avg) for x in arr] )
    return val

def calc_depth(b):
    return 5 - b

def v_opt_rec(xx, b):
    mincost = LARGE_NUM
    n = len(xx)

    # check boundary condition:
    if n < b:
        return LARGE_NUM + 1
    elif b == 1:
        return sse(xx)
    else:  # the general case
        if DEBUG > 1:
            #print('.. BEGIN: input = {!s:<30}, b = {}'.format(xx, b))
            print('..{}BEGIN: input = {!s:<30}, b = {}'.format('  '*calc_depth(b), xx, b))

            
        for t in range(n):
            prefix = x[0 : t+1]
            suffix = x[t+1 : ]
            cost = sse(prefix) + v_opt_rec(suffix, b - 1)
            mincost = min(mincost, cost)

        if DEBUG > 0:
            #print('.. END: input = {!s:<32}, b = {}, mincost = {}'.format(xx, b, mincost))
            print('..{}END: input = {!s:<32}, b = {}, mincost = {}'.format('  '*calc_depth(b), xx, b, mincost))

        return mincost## You can test your implementation using the following code...

import submission as submission
matrix, bins = submission.v_opt_dp(x, num_bins)
print("Bins = {}".format(bins))
print("Matrix =")
for row in matrix:
    print(row)

TypeError: 'NoneType' object is not iterable

In [ ]:
Bins = [[3, 1], [18], [11, 13], [17]]
Matrix =
[-1, -1, -1, 18.666666666666664, 8.0, 0]
[-1, -1, 18.666666666666664, 2.0, 0, -1]
[-1, 18.666666666666664, 2.0, 0, -1, -1]
[4.0, 2.0, 0, -1, -1, -1]